# Heart attack prediction

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
import xgboost as xgb

In [3]:
# 1. Charger le fichier train.parquet
ex = pd.read_parquet('data/extra.parquet')
train = pd.read_parquet('data/train.parquet')

train_data = pd.concat([ex, train])

# Colonnes d'intérêt
columns = ['PERSDOC3', '_CHLDCNT', '_URBSTAT', '_CURECI2', '_RACEG22', 'CHCKDNY2', 'MARITAL', 'DIFFWALK', '_SMOKER3', '_MENT14D', 'USENOW3',
           '_IMPRACE', 'CHECKUP1', 'CHCOCNC1', '_LTASTH1', 'GENHLTH', 'SEXVAR', 'HIVTST7', 'RMVTETH4', '_LLCPWT', '_AGEG5YR', 'HIVRISK5',
           '_RACE1', 'WEIGHT2', 'DRNKANY6', '_PHYS14D', '_DENVST3', 'EMPLOY1', '_METSTAT', 'CVDSTRK3', '_STATE', 'ADDEPEV3', 'HAVARTH4',
           'ALCDAY4', 'MENTHLTH', 'CHCSCNC1', 'BLIND', '_EXTETH3', '_LLCPWT2', '_STRWT', 'TETANUS1', 'CHILDREN', 'DIFFDRES', 'SMOKE100',
           '_PRACE2', 'FLUSHOT7', 'DIABETE4', 'VETERAN3', 'SEQNO', '_RFSMOK3', 'PHYSHLTH', 'HTIN4', '_RAWRAKE', 'MEDCOST1', 'LASTDEN4',
           '_HISPANC', '_RFBMI5', '_DRDXAR2', 'PRIMINSR', 'LCSCTSC1', '_ASTHMS1', 'EXERANY2', '_BMI5CAT', 'QSTVER', '_CASTHM1', 'PNEUVAC4',
           'COVIDPOS', '_BMI5', '_DUALUSE', 'INCOME3', '_RFHLTH', 'HEIGHT3', 'ECIGNOW2', 'SLEPTIM1', 'DEAF', '_HLTHPLN', '_INCOMG1', 'CPDEMO1C',
           'QSTLANG', 'ASTHMA3', 'DIFFALON', 'EDUCA', '_HCVU652', 'DECIDE', 'WTKG3', 'CHCCOPD3', 'RENTHOM1', '_SMOKGRP', 'TARGET']
           

'''
#Identification des colonnes avec le plus de valeurs manquantes
# 1. Calcul du nombre de valeurs manquantes par colonne
dataset_explicatif=train_data.drop(columns=['TARGET', 'ID'])
nan_counts = dataset_explicatif.isna().sum()
# 2. Filtrer les colonnes avec au moins une valeur manquante et trier par ordre décroissant
nan_counts_sorted = nan_counts[nan_counts > 0].sort_values(ascending=False)
# 3. Stocker dans une nouvelle variable
columns_with_nan = nan_counts_sorted
columns_to_drop = columns_with_nan[columns_with_nan > 60000].index.values
#columns_to_drop = np.append(columns_to_drop, ['FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR'])
'''

# Extraction des colonnes
train_data = train_data[columns] #train_data.copy() #train_data[columns] # train_data.drop(columns=columns_to_drop)
train_data.head()

,PERSDOC3,_CHLDCNT,_URBSTAT,_CURECI2,_RACEG22,CHCKDNY2,MARITAL,DIFFWALK,_SMOKER3,_MENT14D,...,ASTHMA3,DIFFALON,EDUCA,_HCVU652,DECIDE,WTKG3,CHCCOPD3,RENTHOM1,_SMOKGRP,TARGET
0,2.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,4.0,2.0,...,2.0,2.0,6.0,9.0,1.0,5670.0,2.0,1.0,4.0,False
1,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,3.0,...,2.0,2.0,5.0,1.0,1.0,13154.0,2.0,2.0,1.0,False
2,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,1.0,...,2.0,2.0,5.0,1.0,2.0,10886.0,2.0,1.0,4.0,False
3,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,1.0,...,2.0,2.0,4.0,1.0,2.0,10659.0,1.0,1.0,2.0,False
4,2.0,1.0,1.0,1.0,1.0,2.0,4.0,2.0,1.0,2.0,...,2.0,2.0,4.0,1.0,2.0,5398.0,2.0,3.0,1.0,False


In [4]:
# 2. Nettoyage des données : remplacer les NaN par la moyenne (ou une autre stratégie)
train_data_cleaned = train_data.apply(lambda col: col.fillna(col.rolling(window=10, min_periods=1).median()), axis=0)

# Séparer les données en variables explicatives (X) et cible (y)
X = train_data_cleaned.drop("TARGET", axis=1)
y = train_data_cleaned["TARGET"]

# Diviser en ensemble d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 3. Entraîner un modèle (XGBoost Classifier)
# Paramètres XGBoost
params = {
    'booster': 'gbtree',                # Type d'algorithme booster
    'learning_rate': 0.4,               # Taux d'apprentissage
    'max_depth': 6,                     # Profondeur maximale des arbres
    'min_child_weight': 1,              # Poids minimum pour les feuilles
    'subsample': 0.8,                   # Fraction des données à utiliser pour chaque arbre
    'colsample_bytree': 0.8,            # Fraction des caractéristiques à utiliser
    'objective': 'binary:logistic',     # Fonction objective pour la classification binaire
    'eval_metric': 'logloss',           # Évaluation avec logloss
    'n_estimators': 1250,                # Nombre d'arbres
    'gamma': 0.1,                       # Réduction de la perte nécessaire pour effectuer une division
}

# Création du modèle XGBoost
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# Évaluer le modèle sur l'ensemble de validation
y_pred = model.predict(X_val)
accuracy = f1_score(y_val, y_pred)
print(f"Accuracy sur l'ensemble de validation: {accuracy:.5f}")

Accuracy sur l'ensemble de validation: 0.24171


In [6]:
# 4. Charger le fichier test.parquet
test_data = pd.read_parquet('data/test.parquet')

# Conserver uniquement les colonnes d'intérêt
columns_test = ['PERSDOC3', '_CHLDCNT', '_URBSTAT', '_CURECI2', '_RACEG22', 'CHCKDNY2', 'MARITAL', 'DIFFWALK', '_SMOKER3', '_MENT14D', 'USENOW3',
           '_IMPRACE', 'CHECKUP1', 'CHCOCNC1', '_LTASTH1', 'GENHLTH', 'SEXVAR', 'HIVTST7', 'RMVTETH4', '_LLCPWT', '_AGEG5YR', 'HIVRISK5',
           '_RACE1', 'WEIGHT2', 'DRNKANY6', '_PHYS14D', '_DENVST3', 'EMPLOY1', '_METSTAT', 'CVDSTRK3', '_STATE', 'ADDEPEV3', 'HAVARTH4',
           'ALCDAY4', 'MENTHLTH', 'CHCSCNC1', 'BLIND', '_EXTETH3', '_LLCPWT2', '_STRWT', 'TETANUS1', 'CHILDREN', 'DIFFDRES', 'SMOKE100',
           '_PRACE2', 'FLUSHOT7', 'DIABETE4', 'VETERAN3', 'SEQNO', '_RFSMOK3', 'PHYSHLTH', 'HTIN4', '_RAWRAKE', 'MEDCOST1', 'LASTDEN4',
           '_HISPANC', '_RFBMI5', '_DRDXAR2', 'PRIMINSR', 'LCSCTSC1', '_ASTHMS1', 'EXERANY2', '_BMI5CAT', 'QSTVER', '_CASTHM1', 'PNEUVAC4',
           'COVIDPOS', '_BMI5', '_DUALUSE', 'INCOME3', '_RFHLTH', 'HEIGHT3', 'ECIGNOW2', 'SLEPTIM1', 'DEAF', '_HLTHPLN', '_INCOMG1', 'CPDEMO1C',
           'QSTLANG', 'ASTHMA3', 'DIFFALON', 'EDUCA', '_HCVU652', 'DECIDE', 'WTKG3', 'CHCCOPD3', 'RENTHOM1', '_SMOKGRP', 'ID']


test_data = test_data[columns_test] #test_data.copy() #test_data[columns_test]

# Nettoyer les données (remplacer les NaN)
test_data_cleaned = test_data.drop(columns=["ID"])
test_data_cleaned = test_data_cleaned.apply(lambda col: col.fillna(col.rolling(window=10, min_periods=1).median()), axis=0)

In [7]:
# 5. Effectuer les prédictions
test_predictions = model.predict(test_data_cleaned)

# Ajouter les prédictions au dataframe original
test_data["TARGET"] = test_predictions

# Exporter les résultats dans un fichier prediction.csv
output_file = 'prediction.csv'
test_data[["ID", "TARGET"]].to_csv(output_file, index=False)

print(f"Les prédictions ont été enregistrées dans le fichier {output_file}.")

Les prédictions ont été enregistrées dans le fichier prediction.csv.
